In [1]:
from dotenv import load_dotenv
import loggingService

from genai.extensions.langchain import LangChainInterface
from genai.model import Credentials
from genai.schemas import GenerateParams

from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import Weaviate

import os
from sentence_transformers import SentenceTransformer
import weaviate

load_dotenv()
logger = loggingService.get_logger()


ModuleNotFoundError: No module named 'transformers'

In [ ]:
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", 'https://workbench-api.res.ibm.com')
# class_name = os.getenv("WEVIATE_CLASS", 'LivrosVectorizer')
print(f"API Key: {api_key}, API Endpoint: {api_endpoint}")
class_name ='Livros'
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question", output_key='answer', return_messages=True)
model_name = os.getenv('MODEL_NAME', 'bigscience/mt0-xxl')
model_name_embedding = os.getenv("MODEL_NAME_EMBEDDING", "sentence-transformers/gtr-t5-large")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(url=weaviate_url,)
embeddings = SentenceTransformer(model_name_embedding)

creds = Credentials(api_key, api_endpoint=api_endpoint)
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=100,
    min_new_tokens=1,
    stream=False,
    temperature=0.5,
    top_k=50,
    top_p=1,
).dict()  # Langchain uses dictionaries to pass kwargs

llm = LangChainInterface(model=model_name, credentials=creds, params=params)
vector_store = Weaviate(client=client, index_name='Livros', text_key='content')


In [ ]:
# vector_store.similarity_search_by_vector(embeddings.encode('por que arthur dent foi despejado?'))
vector_store.similarity_search('por que a casa de arthur dent foi demolida?')


In [ ]:
pt1 = """Responda a pergunta a seguir de forma sucinta usando o contexto e histórico fornecidos. Caso não tenha certeza da resposta siceramente diga que não possui informações suficientes sobre esse tema.

Contexto: {context}

Histórico: {chat_history}

Pergunta: {question}

Resposta:"""
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=pt1,
)

retriver = vector_store.as_retriever(search_kwargs={'score_threshold': 0.8})
# chain_type_kwargs={"prompt": prompt},
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriver, memory=memory,  verbose=True)
# qa = ConversationalRetrievalChain(llm=llm, retriever=retriver, memory=memory,  verbose=True, output_key='Resposta')


In [ ]:
qa.run('por que arthur dent foi despejado?')